In [15]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

## 自定义类
尽管 Keras 提供了很多的常用网络层，但深度学习可以使用的网络层远远不止这些经典的网络层， 对于需要创建自定义逻辑的网络层，可以通过自定义类来实现。在创建自定义网络层类时，需要继承自 layers.Layer 基类； 创建自定义的网络类，需要继承自keras.Model 基类， 这样产生的自定义类才能够方便的利用 Layer/Model 基类提供的参数管理功能，同时也能够与其他的标准网络层类交互使用。

### 1. 自定义网络层

In [34]:
# 首先创建类并继承自 Layer 基类
class MyDense(layers.Layer):
    
    # 自定义网络层
    # inp_dim和outp_dim是输入特征长度和输出特征长度
    def __init__(self, inp_dim, outp_dim):         
        super(MyDense, self).__init__()
        
        # 创建权值张量并添加到类管理列表中，设置为需要优化
        self.kernel = self.add_weight('w', [inp_dim, outp_dim], trainable = True)

In [35]:
# 创建输入为4，输出为3节点的自定义层
net = MyDense(4, 3)
net.variables, net.trainable_variables

([<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[ 0.38847828, -0.4745888 , -0.6601857 ],
         [ 0.7912593 ,  0.16498911, -0.3669604 ],
         [ 0.08805931, -0.49543437, -0.8042041 ],
         [ 0.5179688 ,  0.8228445 ,  0.02395856]], dtype=float32)>],
 [<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[ 0.38847828, -0.4745888 , -0.6601857 ],
         [ 0.7912593 ,  0.16498911, -0.3669604 ],
         [ 0.08805931, -0.49543437, -0.8042041 ],
         [ 0.5179688 ,  0.8228445 ,  0.02395856]], dtype=float32)>])

完成自定义类的初始化工作后，我们来设计自定义类的前项运算逻辑，比如需要完成𝑂=𝑋@𝑊矩阵运算，并通过激活函数。

In [12]:
# 自定义类的前向计算逻辑
def call(self, inputs, training = None):
    out = inputs @ self.kernel
    out = tf.nn.relu(out)
    return out

自定义类的前向运算逻辑需要实现在 call(inputs, training)函数中，其中 inputs 代表输入， 由用户在调用时传入； training 参数用于指定模型的状态： training 为 True 时执行训练模式， training 为 False 时执行测试模式，默认参数为 None，即测试模式。由于全连接层的训练模式和测试模式逻辑一致，此处不需要额外处理。对于部份测试模式和训练模式不一致的网络层，需要根据 training 参数来设计需要执行的逻辑。

### 2. 自定义网络

In [17]:
network = Sequential([
    MyDense(784, 256), 
    MyDense(256, 128), 
    MyDense(128, 10)
])
network.build(input_shape = (None, 28*28))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_7 (MyDense)         multiple                  200704    
_________________________________________________________________
my_dense_8 (MyDense)         multiple                  32768     
_________________________________________________________________
my_dense_9 (MyDense)         multiple                  1280      
Total params: 234,752
Trainable params: 234,752
Non-trainable params: 0
_________________________________________________________________


### 3. 自定义网络类

In [18]:
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
    
        self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 10)
        
    def call(self, inputs, training = None):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
    